##This is the cool project Airline Schedule

###Construct Graph for routes, Node should have 6, airport with x gates has x Node, each node has ###'Ground_Time' attribute, make it 2 minutes more than min. DAL1 and DAL2 has no edge, same as HOU1,HOU2 and HOU3


1. 360(600 morning)<=DTM<ATM<=1320(2200 night)

2. AUS->DAL  50 Edge_LEN
   DAL->AUS  50
   AUS->HOU  45
   HOU->AUS  45
   DAL->HOU  65
   HOU->DAL  65

3. AUS1  GTM_END - GTM_START >= 25 or GTM_END, GTM_START are 0 
   DAL1  GTM_END - GTM_START >= 30
   DAL2  GTM_END - GTM_START >= 30
   HOU1  GTM_END - GTM_START >= 35
   HOU2  GTM_END - GTM_START >= 35
   HOU3  GTM_END - GTM_START >= 35

4. DTM < GTM_START
   ATM > GTM_END

5. DTM = ATM + (GTM_END-GTM_START)

6. ATM =  DTM + Edge_LEN

7. AUS T1
   DAL T5
   DAL T6
   HOU T2
   HOU T3
   HOU T4

8. Use Random to pick next node
   
   https://stackoverflow.com/questions/306400/how-to-randomly-select-an-item-from-a-list
   
   For i in Node List
    STOP_EOD: Record end day occupied airport, so next route won't take it
             [AUS1, DAL1, DAL2, HOU1[1,3]]
             
    While Sum(Edge.Time + Node.Time) <= 1320 - 360
        
        Pick candidate list,  DAL1-{AUS1, HOU1, HOU2, HOU3}
        
    --Check Last Node, make sure it's not in STOP_EOD List
    IF last node in STOP_EOD, then accept it and remove this node from STOP_EOD, so next route        won't take it
    ELSE
       Remove last node from route, pick last 2nd node's candidate list again.
       
    Return node dictionary with total time,{time:[1,2,3,4,5]}
    
    Convert this results to fly schedule, replace DAL1/DAL2 to DAL, etc.
    
    
    
        

In [8]:
import networkx as nx
import random
import re
from pandas import DataFrame

#define minutes to military time
def convert_military_time (var): 
    return var//60 * 100 + var % 60

#define get to pull value from optimal_schedule
def get_var(var):
    variable = []
    
    if var == 'd_time':
        for i in Optimal_Schedule:
            value = convert_military_time(i[var])      
            variable.append(value)
    elif var == 'a_time':
        for i in Optimal_Schedule:
            value = convert_military_time(i[var])      
            variable.append(value)
    else:
        for i in Optimal_Schedule:
            value = i[var] 
            variable.append(value)
           
    return variable

#Initialize graph
routes = nx.Graph()

#ground_time as attribute
routes.add_node("AUS1", ground_time = 27)
routes.add_node("DAL1", ground_time = 32)
routes.add_node("DAL2", ground_time = 32)
routes.add_node("HOU1", ground_time = 37)
routes.add_node("HOU2", ground_time = 37)
routes.add_node("HOU3", ground_time = 37)

#fly_time as weight for edge
routes.add_edge("AUS1","DAL1", fly_time = 50)
routes.add_edge("DAL1","AUS1", fly_time = 50)
routes.add_edge("AUS1","DAL2", fly_time = 50)
routes.add_edge("DAL2","AUS1", fly_time = 50)

routes.add_edge("AUS1","HOU1", fly_time = 45)
routes.add_edge("HOU1","AUS1", fly_time = 45)
routes.add_edge("AUS1","HOU2", fly_time = 45)
routes.add_edge("HOU2","AUS1", fly_time = 45)
routes.add_edge("AUS1","HOU3", fly_time = 45)
routes.add_edge("HOU3","AUS1", fly_time = 45)

routes.add_edge("HOU1","DAL1", fly_time = 65)
routes.add_edge("DAL1","HOU1", fly_time = 65)
routes.add_edge("HOU2","DAL1", fly_time = 65)
routes.add_edge("DAL1","HOU2", fly_time = 65)
routes.add_edge("HOU3","DAL1", fly_time = 65)
routes.add_edge("DAL1","HOU3", fly_time = 65)

routes.add_edge("HOU1","DAL2", fly_time = 65)
routes.add_edge("DAL2","HOU1", fly_time = 65)
routes.add_edge("HOU2","DAL2", fly_time = 65)
routes.add_edge("DAL2","HOU2", fly_time = 65)
routes.add_edge("HOU3","DAL2", fly_time = 65)
routes.add_edge("DAL2","HOU3", fly_time = 65)

#final_list is a list with dictionary entry,use this to build final schedule
def transform (candidate_list, final_list, graph, tail_num):
       
    stage_list = []
    regex = re.compile('\d+') #Remove all number, so only airport code exists 
    
    for i in range(len(candidate_list) - 1):
        stage_list .append({})
                
        stage_list[i]['tail_num'] = tail_num
        stage_list[i]['org'] = regex.sub('',candidate_list[i])
        stage_list[i]['dest'] = regex.sub('',candidate_list[i+1])
        
        if i == 0: #This is important for cal, 1st flight all day
            stage_list[i]['d_time'] = 360
        else:
            stage_list[i]['d_time'] = stage_list[i-1].get('a_time') + graph.node[candidate_list[i-1]]['ground_time']
        
        stage_list[i]['a_time'] = int(stage_list[i].get('d_time')) + graph[candidate_list[i]][candidate_list[i+1]]['fly_time']
        
    
    return final_list + stage_list

#Final Schedule will look like
#[{'tail_num':'T1', 'org':'DAL', 'dest':'AUS', 'd_time':360, 'a_time':410}, {},{}]
Optimal_Schedule = []

#Build airport node list
node_nm = dict(enumerate(['HOU3','DAL1','AUS1','HOU1', 'DAL2', 'HOU2']))
secure_random = random.SystemRandom()

#STOP_EOD: Record end day occupied airport, so next route won't take it
STOP_EOD = ['AUS1', 'DAL1', 'DAL2', 'HOU1','HOU2','HOU3']

#Sum_Time is to calculate Edge.Time + Node.Time, it must be <= 1320 - 360 = 960 minutes
sum_time = 0

#Initialize parameter
route_list = []

for key, elem in node_nm.items():

    route_list.clear() 
    route_list.append(elem)
    sum_time = 0
    
    while sum_time <= 960:
        pick_node = secure_random.choice(node_nm)
        #print(elem[0], pick_node[0])

        
        if elem[0] != pick_node[0]:
            if (sum_time + routes[elem][pick_node]['fly_time']) <=960:
                sum_time += routes[elem][pick_node]['fly_time']
                sum_time += routes.node[pick_node]['ground_time']
                route_list.append(pick_node)
                #so next pick won't choose previous same airport, like DAL1 and DAL2 is bad pick
                elem = pick_node
            elif (sum_time + routes[elem][pick_node]['fly_time']) > 960:
                pick_node = route_list[-1] # if don't pass last existing node to pick_node, below logic will fail
                break
                

    #End of while loop
    
    #Logic to adjust last node/stop of route due to requirement, use STOP_EOD list
    if pick_node in STOP_EOD:
        STOP_EOD.remove(pick_node)
    else:
        route_list = route_list[:-1]
        #Chooose another one from STOP_EOD list
        pick_node = secure_random.choice(STOP_EOD)
        
        #Check if last airport of route_list is same to pick_node or not
        if route_list[-1][0] != pick_node[0]:
            route_list.append(pick_node)
            STOP_EOD.remove(pick_node)
        else:#if same, replace last airport with pick_node(fly_time still same)so last 2 nodes are still different
            route_list = route_list[:-1]
            route_list.append(pick_node)
            STOP_EOD.remove(pick_node)
            
                
    #End of adjustment code block
    
    #Here we use transform func, pass it to Optimal_Schedule, key+1 is tailnumber for plane
    #print ("After cleaning",route_list, sum_time)
    #print ("\n")
    Optimal_Schedule = transform(route_list, Optimal_Schedule, routes, 'T'+str(key+1))
    
    secure_random.shuffle
    
#print(Optimal_Schedule)


#Once Optimal_Schedule is done, use below code to generate file
#use function get_var
tail_num = get_var('tail_num')
org = get_var('org')
dest = get_var ('dest')
d_time = get_var ('d_time')
a_time = get_var ('a_time')

data = {'tail_number':tail_num, 'origin':org, 'destination':dest, 'departure_time':d_time, 'arrival_time':a_time}
frame = DataFrame(data)
frame.to_csv('flight_schedule.csv', mode = 'w', index=False, columns=['tail_number', 'origin', 'destination', 'departure_time','arrival_time'])

[{'dest': 'DAL', 'tail_num': 'T1', 'a_time': 425, 'org': 'HOU', 'd_time': 360}, {'dest': 'HOU', 'tail_num': 'T1', 'a_time': 527, 'org': 'DAL', 'd_time': 462}, {'dest': 'AUS', 'tail_num': 'T1', 'a_time': 604, 'org': 'HOU', 'd_time': 559}, {'dest': 'HOU', 'tail_num': 'T1', 'a_time': 686, 'org': 'AUS', 'd_time': 641}, {'dest': 'DAL', 'tail_num': 'T1', 'a_time': 778, 'org': 'HOU', 'd_time': 713}, {'dest': 'HOU', 'tail_num': 'T1', 'a_time': 880, 'org': 'DAL', 'd_time': 815}, {'dest': 'DAL', 'tail_num': 'T1', 'a_time': 977, 'org': 'HOU', 'd_time': 912}, {'dest': 'HOU', 'tail_num': 'T1', 'a_time': 1079, 'org': 'DAL', 'd_time': 1014}, {'dest': 'AUS', 'tail_num': 'T1', 'a_time': 1156, 'org': 'HOU', 'd_time': 1111}, {'dest': 'DAL', 'tail_num': 'T1', 'a_time': 1243, 'org': 'AUS', 'd_time': 1193}, {'dest': 'HOU', 'tail_num': 'T2', 'a_time': 425, 'org': 'DAL', 'd_time': 360}, {'dest': 'DAL', 'tail_num': 'T2', 'a_time': 522, 'org': 'HOU', 'd_time': 457}, {'dest': 'AUS', 'tail_num': 'T2', 'a_time': 6

###So now the only defect is...last two nodes might have same airports, but we can clean it later.
###Now the logic is how to use this route list and generate Optimal_schedule below

1. one tail_num <-> one route_list
2. d_time all start from 360, then calcuate a_time, ground_time, etc



In [15]:
from pandas import DataFrame

Optimal_Schedule = [
    {'tail_num':'T1', 'org':'DAL', 'dest':'AUS', 'd_time':360, 'a_time':410},
    {'tail_num':'T1', 'org':'AUS', 'dest':'HOU', 'd_time':600, 'a_time':645}
]

#define minutes to military time
def convert_military_time (var): 
    return var//60 * 100 + var % 60

#define
def get_var(var):
    variable = []
    
    if var == 'd_time':
        for i in Optimal_Schedule:
            value = convert_military_time(i[var])      
            variable.append(value)
    elif var == 'a_time':
        for i in Optimal_Schedule:
            value = convert_military_time(i[var])      
            variable.append(value)
    else:
        for i in Optimal_Schedule:
            value = i[var] 
            variable.append(value)
           
    return variable

#use function
tail_num = get_var('tail_num')
org = get_var('org')
dest = get_var ('dest')
d_time = get_var ('d_time')
a_time = get_var ('a_time')

data = {'tail_number':tail_num, 'origin':org, 'destination':dest, 'departure_time':d_time, 'arrival_time':a_time}
frame = DataFrame(data)
frame.to_csv('flight_schedule.csv', mode = 'w', index=False, columns=['tail_number', 'origin', 'destination', 'departure_time','arrival_time'])

In [26]:
import random

node_nm = dict(enumerate(['AUS1', 'DAL1', 'DAL2', 'HOU1','HOU2','HOU3']))
secure_random = random.SystemRandom()
#print (node_nm.keys())
#print (secure_random.choice(node_nm))

#STOP_EOD: Record end day occupied airport, so next route won't take it
STOP_EOD = ['AUS1', 'DAL1', 'DAL2', 'HOU1','HOU2','HOU3']

#Sum_Time is to calculate Edge.Time + Node.Time, it must be <= 1320 - 360 = 960 minutes
sum_time = 0

for key, elem in node_nm.items():
    print (key, elem)
    route_list= [elem]
    
    #While sum_time <= 960
    pick_node = secure_random.choice(node_nm)
    
    route_list.append(pick_node)
    print(route_list)
    #and (sum_time + routes[elem][pick_node]['fly_time'] + routes.node[pick_node]['ground_time'] <= 960)
    
    
    
        
        
        
    
    



0 AUS1
['AUS1', 'AUS1']
1 DAL1
['DAL1', 'HOU3']
2 DAL2
['DAL2', 'HOU3']
3 HOU1
['HOU1', 'HOU3']
4 HOU2
['HOU2', 'HOU2']
5 HOU3
['HOU3', 'DAL2']
